![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [153]:
import pandas as pd
import numpy as np

marketing = pd.read_csv('bank_marketing.csv')
client2 = pd.read_csv('client.csv')
campaign2 = pd.read_csv('campaign.csv')
economics2 = pd.read_csv('economics.csv')

In [154]:
print(marketing.columns)
print(client2.columns)
print(campaign2.columns)
print(economics2.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')
Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'campaign_id', 'last_contact_date'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


In [155]:
client = marketing[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan']]

In [156]:
campaign = marketing[['client_id', 'contact', 'duration', 'pdays',
       'previous', 'poutcome', 'campaign','y', 'month', 'day']]

In [157]:
economics = marketing[['client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m','nr_employed']]

In [158]:
client.rename(columns = {'client_id':'id'}, inplace = True)

In [159]:
campaign.rename(columns = {'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}, inplace = True)

In [160]:
economics.rename(columns = {'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'}, inplace = True)

In [161]:
marketing['education'] = marketing['education'].str.replace('.','_')

In [162]:
marketing['education'] = marketing['education'].replace('unknown',np.nan)

In [163]:
client['job'] = client['job'].str.replace('.','')

In [164]:
def convertir_a_binario(valor):
    if valor == 'success':
        return 1
    if valor == 'failure':
        return 0
    if valor == 'nonexistent':
        return np.nan
    else:
        return valor

In [165]:
campaign['previous_outcome'] = campaign['previous_outcome'].map(convertir_a_binario)

In [166]:
campaign['campaign_id'] = 1

In [167]:
campaign['month'].apply(str)

0        may
1        may
2        may
3        may
4        may
        ... 
41183    nov
41184    nov
41185    nov
41186    nov
41187    nov
Name: month, Length: 41188, dtype: object

In [168]:
year = '2022'
campaign['last_contact_date'] = pd.to_datetime(campaign.assign(year=year)[['year','month', 'day']].astype(str).agg('-'.join, axis=1), format="%Y-%b-%d")

In [169]:
print(client2.columns)
print(client.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')


In [170]:
print(campaign2.columns)
print(campaign.columns)

Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'campaign_id', 'last_contact_date'],
      dtype='object')
Index(['client_id', 'contact', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'number_contacts',
       'campaign_outcome', 'month', 'day', 'campaign_id', 'last_contact_date'],
      dtype='object')


In [171]:
campaign = campaign[['client_id', 'number_contacts', 'contact_duration', 'pdays', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'campaign_id', 'last_contact_date']]
print(campaign.columns)

Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'campaign_id', 'last_contact_date'],
      dtype='object')


In [172]:
print(economics2.columns)
print(economics.columns)

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx',
       'euribor_three_months', 'number_employed'],
      dtype='object')


In [173]:
economics = economics[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months', 'number_employed']]
print(economics.columns)

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


In [174]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [175]:
client_table = '''
CREATE TABLE client ( 
id SERIAL PRIMARY KEY,
age INTEGER,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOLEAN,
housing BOOLEAN,
loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER;
'''

In [176]:
campaign_table = '''
CREATE TABLE campaign ( 
campaign_id SERIAL PRIMARY KEY,
number_contacts INTEGER,
contact_duration INTEGER,
pdays INTEGER,
previous_campaign_contacts INTEGER,
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE
);
ALTER TABLE campaign
ADD COLUMN client_id SERIAL REFERENCES client (id);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER;
'''

In [177]:
economics_table = '''
CREATE TABLE economics ( 
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT
);
ALTER TABLE economics
ADD COLUMN client_id SERIAL REFERENCES client (id);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER;
'''

In [178]:
#client = "\copy client from 'client.csv' DELIMITER ',' CSV HEADER"

In [179]:
#campaign = "\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER"

In [180]:
#economic = "\copy economic from 'economic.csv' DELIMITER ',' CSV HEADER"